<a href="https://colab.research.google.com/github/jessicanadalete/clustering_fidelity/blob/main/Clustering_Fidelity_Project_Cycle2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0.0 Import Libraries
-----

In [ ]:
pip install inflection

In [ ]:
pip install umap-learn

In [ ]:
import inflection
import datetime
import re
import pandas       as pd
import numpy        as np
import seaborn      as sns
import umap.umap_   as umap

from sklearn              import cluster        as c
from sklearn              import metrics        as m
from yellowbrick.cluster  import KElbowVisualizer
from yellowbrick.cluster  import SilhouetteVisualizer
from matplotlib           import pyplot         as plt
from plotly               import express        as px

### 0.1 Load Dataset

In [ ]:
data_raw = pd.read_csv('/content/Ecommerce.csv', encoding='unicode_escape')

## 1.0 Data Description

---






In [ ]:
df1 = data_raw.copy()

###1.1 Data Dimension

In [ ]:
df1.shape

(541909, 9)

In [ ]:
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Unnamed: 8
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom,NaN
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom,NaN
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN


In [ ]:
df1 = df1.drop('Unnamed: 8', axis=1)

In [ ]:
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom


### 1.2 Rename Columns

In [ ]:
df1.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [ ]:
cols_old = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country']

snakecase = lambda x: inflection.underscore(x)
cols_new = list(map(snakecase, cols_old))

#rename columns
df1.columns = cols_new

In [ ]:
df1.columns

Index(['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'country'],
      dtype='object')

### 1.3 Data Types

In [ ]:
df1.dtypes

,0
invoice_no,object
stock_code,object
description,object
quantity,int64
invoice_date,object
unit_price,float64
customer_id,float64
country,object


### 1.4 Check NAs

In [ ]:
df1.isna().sum()

,0
invoice_no,0
stock_code,0
description,1454
quantity,0
invoice_date,0
unit_price,0
customer_id,135080
country,0


### 1.5 Remove/Replace NAs

In [ ]:
df1 = df1.dropna(subset=['description','customer_id'])

In [ ]:
df1.shape

(406829, 8)

In [ ]:
df1.isna().sum()

,0
invoice_no,0
stock_code,0
description,0
quantity,0
invoice_date,0
unit_price,0
customer_id,0
country,0


###1.6 Change Types

In [ ]:
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'],format='%d-%b-%y')

In [ ]:
df1['customer_id'] = df1['customer_id'].astype(int)

In [ ]:
df1['unit_price'] = df1['unit_price'].astype(float)

In [ ]:
df1.dtypes

,0
invoice_no,object
stock_code,object
description,object
quantity,int64
invoice_date,datetime64[ns]
unit_price,float64
customer_id,int64
country,object


###1.7 Statistic Description

In [ ]:
num_attributes = df1.select_dtypes(include=['int64', 'float64'])
cat_attributes = df1.select_dtypes(exclude=['int64', 'float64','datetime64[ns]'])

###1.7.1 Numerical Attributes

In [ ]:
#central tendency - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

#dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(min)).T
d3 = pd.DataFrame(num_attributes.apply(max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T #range
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

#concatenate
m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,quantity,-80995.0,80995.0,161990.0,12.061303,5.00,248.693064,0.182663,94317.563673
1,unit_price,0.0,38970.0,38970.0,3.460471,1.95,69.315077,452.219019,246924.542988
2,customer_id,12346.0,18287.0,5941.0,15287.690570,15152.00,1713.598197,0.029835,-1.179982


###<font color='red'>1.7.1.1 Numerical Attributes - Investigating </font>
1. Negative values in Quantity (devolution?)
2. Unity price equal to zero (sale?)




###1.7.2 Categorical Attributes

In [ ]:
cat_attributes.head()

,invoice_no,stock_code,description,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom
1,536365,71053,WHITE METAL LANTERN,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,United Kingdom


In [ ]:
#cat_attributes['invoice_no'].astype(int)
len(cat_attributes.loc[cat_attributes['invoice_no'].apply(lambda x: bool (re.search('[^0-9]+', x))), 'invoice_no'].drop_duplicates())

3654

In [ ]:
len(cat_attributes.loc[cat_attributes['stock_code'].apply(lambda x: bool (re.search('[^0-9]+', x))), 'stock_code'].drop_duplicates())

886

In [ ]:
df1.loc[cat_attributes['stock_code'].apply(lambda x: bool (re.search('^[A-Za-z]+$', x))), 'stock_code'].unique()

array(['POST', 'D', 'M', 'PADS', 'DOT', 'CRUK'], dtype=object)

## 2.0 Feature Engineering
-----

In [ ]:
df2 = df1.copy()

In [ ]:
df2.head()

In [ ]:
df2.columns

###2.1 Feature Creation

In [ ]:
#data reference - creating a dataframe with unique values
df_ref = df2.drop(['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'country'], axis=1).drop_duplicates(ignore_index=True) #extrating customer_ids and reseting index
df_ref.shape

In [ ]:
df_ref.head()

In [ ]:
# creating Gross Revenue variable (Faturamento)
df2['gross_revenue'] = df2['quantity'] * df2['unit_price']

# creating Monetary (value spent by customer)
df_monetary = df2[['customer_id','gross_revenue']].groupby('customer_id').sum().reset_index()

# creating Recency variable (last date purchase)
df_recency = df2[['customer_id','invoice_date']].groupby('customer_id').max().reset_index() # getting the last date purchase by customer
df_recency['recency_days'] = (df2['invoice_date'].max() - df_recency['invoice_date']).dt.days # calculating and extrating days quantity
df_recency = df_recency[['customer_id','recency_days']].copy() #selecting columns

# creating Frquency (purchase quantities)
df_frequency = df2[['customer_id','invoice_no']].drop_duplicates().groupby('customer_id').count().reset_index()

In [ ]:
#creating avg ticket (value average by custumer)
df_avg_ticket = df2[['customer_id','gross_revenue']].groupby('customer_id').mean().reset_index().rename(columns={'gross_revenue':'avg_ticket'})
df_avg_ticket.head()

In [ ]:
df2['invoice_date'].max()

In [ ]:
df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')
df_ref = pd.merge(df_ref, df_recency, on='customer_id', how='left')
df_ref = pd.merge(df_ref, df_frequency, on='customer_id', how='left')
df_ref = pd.merge(df_ref, df_avg_ticket, on='customer_id', how='left')

In [ ]:
df_ref.head()

In [ ]:
df_ref.isna().sum()

## 3.0 Data Filtering
-----

In [ ]:
df3 = df_ref.copy()

## 4.0 EDA (Exploratory Data Analysis)
-----

In [ ]:
df4 = df3.copy()

## 5.0 Data Preparation
-----

In [ ]:
df5 = df4.copy()

## 6.0 Feature Selection
-----

In [ ]:
df6 = df5.copy()

## 7.0 Hyperparameter Fine-Tunning
-----

In [ ]:
df7 = df6.copy()

In [ ]:
X = df7.drop(columns = ['customer_id']) #droping variable without information

In [ ]:
X.head()

In [ ]:
clusters = [2,3,4,5,6,7]

###7.1 Within-Cluster Sum of Square (WSS)

In [ ]:
#model defining and analyzing Elbow
kmeans = KElbowVisualizer(c.KMeans(), k=clusters, timing=False)
kmeans.fit(X)
kmeans.show()

###7.2 Silhouette Score

In [ ]:
#model defining and analyzing Elbow
kmeans = KElbowVisualizer(c.KMeans(), k=clusters, metric='silhouette', timing=False)
kmeans.fit(X)
kmeans.show()

##7.3 Silhouette Analysis

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(25, 18))

# transform 2d matrix to a list
axes = axes.flatten()

for i, k in enumerate(clusters):
    km = c.KMeans(n_clusters=k, init='random', n_init=10, max_iter=100, random_state=42)
    visualizer = SilhouetteVisualizer(km, color='yellowbrick', ax=axes[i])
    visualizer.fit(X)
    visualizer.finalize()

In [ ]:
# fig, ax = plt.subplots (3,2,figsize=(25,18))
# for k in clusters:
#   km = c.KMeans(n_clusters=k, init='random', n_init=10, max_iter=100, random_state=42)
#   q, mod = divmod(k,2)
#   visualizer = SilhouetteVisualizer(km, color='yellowbricks', ax=ax[q-1][mod])
#   visualizer.fit(X)
#   visualizer.finalize()

## 8.0 Model Training
-----

###8.1 K-means Model

In [ ]:
# model definition
k = 4
kmeans = c.KMeans (init='random', n_clusters=k,n_init=10, max_iter=300, random_state=42)

# model training
kmeans.fit(X)

#clustering
labels = kmeans.labels_

###8.2 Cluster Validation

In [ ]:
# WSS
print('WSS value {}'.format(kmeans.inertia_))

#Silhouette Score
print('Silhouette Score {}'.format(m.silhouette_score(X, labels, metric='euclidean')))

## 9.0 Cluster Analysis
-----

In [ ]:
df9 = df7.copy()

In [ ]:
df9['cluster'] = labels

###9.1 Visualization Inspection

In [ ]:
fig = px.scatter_3d(df9, x='recency_days', y='invoice_no', z='gross_revenue', color='cluster')
fig.show()

### 2D Plot

In [ ]:
df_viz = df9.drop(columns='customer_id', axis=1)
sns.pairplot(df_viz, hue='cluster')

### UMAP - t-SNE (Manifold - Learning by topology)

In [ ]:
reducer = umap.UMAP(n_neighbors=50, random_state=42)
embedding = reducer.fit_transform(X) #projecting space

df_viz['embedding_x'] = embedding[:,0]
df_viz['embedding_y'] = embedding[:,1]

#plot UMAP
sns.scatterplot(x='embedding_x', y='embedding_y', hue='cluster',
                palette=sns.color_palette('hls', n_colors=df_viz['cluster'].nunique()),
                data=df_viz)

###9.2 Cluster Profile

In [ ]:
#Number of customer by cluster
df_cluster = df9[['customer_id', 'cluster']].groupby('cluster').count().reset_index()
df_cluster['percentage'] = 100*(df_cluster['customer_id']/df_cluster['customer_id'].sum())
df_cluster

In [ ]:
# Gross Revenue Average
df_gr_avg = df9[['gross_revenue', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_gr_avg, how='inner', on='cluster')
df_cluster

In [ ]:
# Recency Days Average
df_gr_avg = df9[['recency_days', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_gr_avg, how='inner', on='cluster')
df_cluster

In [ ]:
# Invoice_no Average
df_gr_avg = df9[['invoice_no', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_gr_avg, how='inner', on='cluster')
df_cluster

In [ ]:
#Ticket Average
ticket_avg = df9[['avg_ticket', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, ticket_avg, how='inner', on='cluster')
df_cluster

##Report
### Cluster 1 (Potential Insider):
- Customers: 6 (0.14% of total customer)
- Recency Average: 7 days
- Purchase Average: 89 purchases
- Gross Revenue Average: $182,182.00

### Cluster 2:
- Customers: 13 (0.7% of total customer)
- Recency Average: 14 days
- Purchase Average: 53 purchases
- Gross Revenue Average: $40,543.50

### Cluster 3:
- Customers: 4335 (99% of total customer)
- Recency Average: 92 days
- Purchase Average: 5 purchases
- Gross Revenue Average: $1,372.50


## 10.0 Deploy to Production
-----